<h3>This code/notebook calculates the histogram of given graphs and only compress graph of 3,4,5 cycle and then recalcualte them and then train random forest using k-fold. Only histogram are used for training in this code

In [17]:
import igraph, time, copy, os, pickle, random
from scipy.io import loadmat
from collections import Counter
import pickle, pathlib
import numpy as np
import pandas as pd
import base
import sys
%matplotlib inline

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from tqdm import tqdm, tqdm_notebook
import matplotlib.pyplot as plt
from IPython.core.display import display, SVG, HTML

# functions

In [2]:
def return_grid_search_random_forest():
    return RandomForestClassifier(n_estimators=500, 
                        min_samples_leaf=1, 
                            min_weight_fraction_leaf=0.0, max_features=None, max_leaf_nodes=None, bootstrap=True, 
                                oob_score=False, n_jobs=8, random_state=666, verbose=0, warm_start=False,
                                    max_depth = None,class_weight="balanced_subsample")

In [3]:
def return_connected_components(n=5, imp_feature=False):
    '''
    returns family of graphs as igraph objects
    you can specify the number of connected components graphs here 
    imp_feature: if true will include only paths, cycle, stars and cliques
    '''                            'g_6_47', 'g_6_58', 'g_4_2', 'g_5_21', 'g_6_24', 'g_6_47']
    included_list = []
    CONNECTED_COMPONENTS_PATH   = 'graphs'
    LEN_OF_CONNECTED_COMPONENTS =  n
    dirs = []
    connected_graphs_components = {}
    for i in os.listdir(CONNECTED_COMPONENTS_PATH):
        if LEN_OF_CONNECTED_COMPONENTS >= int(i[3:]):
            dirs.append(i)


    for path in dirs:
        for count,file in enumerate(os.listdir(os.path.join(CONNECTED_COMPONENTS_PATH, path))): 
            file_path = os.path.join(CONNECTED_COMPONENTS_PATH, path, file)
            if file_path.endswith('.txt'):
                count = ''.join(x for x in file if x.isdigit())
                #if 'g_'+path[3:]+'_'+str(count) in included_list:
                connected_graph = igraph.load(file_path).as_undirected()
                connected_graphs_components['g_'+path[3:]+'_'+str(count)] = connected_graph
    
    if imp_feature and n==6:
        arr = ['g_3_1', 'g_3_2', 'g_4_2', 'g_4_1', 'g_4_4', 'g_4_6',
              'g_5_14', 'g_5_1', 'g_5_21', 'g_5_9', 'g_6_1','g_6_19','g_6_47', 'g_6_112']
        connected_graphs_components_ = {i:connected_graphs_components[i] for i in arr}
        return connected_graphs_components_
    return connected_graphs_components          

In [4]:
def return_hist_and_contracted_graphs(graph1, connected_graphs_components):
    
    '''computes the histogram of a dataset containing graphs for given connected components'''
    
    contracted_graphs = {}
    dict_hist         = {}

    for name, k_con_graph in connected_graphs_components.items():
        v, len_v = return_disconnected_components(graph1, k_con_graph, name)
        if len_v>0:
            cntrct_graph = return_contracted_graph(graph1, list(v.values())[0])
            contracted_graphs[name] = cntrct_graph
            dict_hist[name]         = len_v
    return dict_hist, contracted_graphs 

def return_contracted_graph(graph1, vertices, layer_val = 2):
    '''contract grahs based on the given vertices
    layer_val: on which layer we are doing contracting the graph'''
    graph = copy.copy(graph1)
    merge_v = graph.vs.indices
    v_arr   = []
    for v in vertices:
        if len(v)>0:
            merge_v = [v[0] if x in v else x for x in merge_v]
            v_arr.append(v[0])
    graph.contract_vertices(merge_v)
    #simplify double edges to single edges and delete nodes that have no edge
    graph.vs[v_arr]['layer'] = layer_val
    graph.simplify(), graph.vs.select(_degree=0).delete()
    graph.vs['idx']          = [str(i) for i in graph.vs.indices]
    return graph

def return_disconnected_components(graph1, connected_graphs_component, name, HOGN=False, layer_val = 2):
    '''
    finds the given family of graph in a given graph
    HOGN: defines the verbosity level all verbosities are calculated by default
    return mappings and histogram of family of subgraphs'''
    graph = copy.copy(graph1)
    disconnected_comp = {}
    disconnected_comp[name] = []
    HOGN_var = 0
        
    index = graph.subisomorphic_vf2(connected_graphs_component, return_mapping_21=True)[2]
    if len(index)>0:
        disconnected_comp[name].append(graph.vs[index]['idx'])
        if HOGN and layer_val in graph.vs[index]['layer']:
            HOGN_var +=1
        graph.delete_vertices(index)
    while len(index)>0:
        index = graph.subisomorphic_vf2(connected_graphs_component, return_mapping_21=True)[2]
        if len(index)>0:
            disconnected_comp[name].append(graph.vs[index]['idx'])
            if HOGN and layer_val in graph.vs[index]['layer']:
                HOGN_var +=1
            graph.delete_vertices(index)
    disconnected_comp[name] = [list(map(int,i)) for i in disconnected_comp[name]]
    if HOGN:
        return disconnected_comp, len(list(disconnected_comp.values())[0]), HOGN_var
    else:
        return disconnected_comp, len(list(disconnected_comp.values())[0])

def return_hist_and_contracted_graphs_of_graphs(graph1, connected_graphs_components, layer_val=2, save=True):
     '''
    computes the histogram of a dataset containing laready contracted graphs input as dict for given connected components
    can be extended to multiple layers'''
    contracted_graphs= {}
    dict_hist        = {}
    range_val        = 6
    new_dict         = {}
    #contracted_graphs[key] = {}
    for idx, graph in graph1.items():
        new_dict = {idx+"_"+str(i)+'_'+str(layer_val)+"_"+str(j):0 for j in range(0,range_val+1) for i in range(3,range_val+1)}
        for name, k_con_graph in connected_graphs_components.items():
            v, len_v, HOGN_var = return_disconnected_components(graph, k_con_graph, name, HOGN = True, layer_val=layer_val)
            for i in list(v.values())[0]:
                new_dict[idx+"_"+name[2:3]+"_"+str(layer_val)+"_"+str(graph.vs[i]['layer'].count(layer_val))] += 1 
            if save:
                contracted_graphs[idx+"-"+name] = return_contracted_graph(graph, list(v.values())[0])
            dict_hist[idx+"-"+name+"-HOGN"] = HOGN_var
            dict_hist[idx+"-"+name]         = len_v
    return  {**dict_hist, **new_dict} , contracted_graphs



In [5]:
def scale_and_drop_col(data_train, data_test, frac=0.9, scale = True):
    data_train      = data_train[data_train.columns[((data_train == 0).astype(int).sum(axis=0)/data_train.shape[0])<frac]] # keep col with 0 less that 90%
    data_test       = data_test[data_train.columns]
    #columns that are in float or not integer
    scaler       = MinMaxScaler()
    #scale integer column from range 0-1
    scaled_columns = None
    if scale:
        scaled_columns = data_train.columns
        scaled_train   = scaler.fit_transform(data_train.values)
        data_train     = pd.DataFrame(scaled_train, columns = scaled_columns)
        data_test      = pd.DataFrame(scaler.transform(data_test[scaled_columns].values), columns = scaled_columns)
    #concat the scaled integer columns with float columns
    
    return data_train, data_test, scaler, scaled_columns

In [6]:
def return_dataset(file_name):
     '''
    downloads the data from graph kernel website only limited graphs are available
    returns graphs as igraph object, their attribue and class label'''
    #i = 'G_nci1'
    dd = base.fetch_dataset(file_name)
    graph_list = []
    node_attr = []
    for gg in dd.data:
        v = set([i[0] for i in gg[0]]).union(set([i[1] for i in gg[0]]))
        g_ = igraph.Graph()
        g_.add_vertices([str(i) for i in v])
        g_.add_edges([(str(i[0]), str(i[1])) for i in gg[0]])
        g_.simplify()
        graph_list.append(g_)
        g_.vs['idx'] = [str(i) for i in g_.vs.indices]
        node_attr.append(gg[1])
    data_y = dd.target
    return graph_list, data_y, node_attr

def set_node_attrs(graph_list, data_nodes):
    for i in range(len(graph_list)):
        for key, value in data_nodes[i].items():
            try:
                graph_list[i].vs.find(str(key))['attr'] = value
            except ValueError:
                print('Lonely node at',i,key)

In [7]:
def return_basic_stat(graph_list):
     '''
    returns edge and vertex count'''
    data_k = {}
    for idx , graph_ in enumerate(graph_list):
        data_k[idx] = {}
        data_k[idx]['n_V']    = graph_.vcount() 
        data_k[idx]['n_e']    = graph_.ecount()
    return data_k

def return_basic_stat_contracted(graph_cn):
    '''returns edge and vertex count of contract graphs given in a dict'''
    data_k = {}
    for idx , graph_ in graph_cn.items():
        data_k[idx+"_N_v"]    = graph_.vcount() 
        data_k[idx+"_N_e"]    = graph_.ecount()
        #data_k[idx]['avg_path_len'] =  graph_.average_path_length()
        
        #data_k[idx]['conn_comp']    =  len(graph_.components())
    return data_k

In [8]:
range_val = 6

# Calculate Histogram (change name of dataset here)

Execute the code below to get the graph descriptor of the given dataset.
It will generate 10 random motif based graph descriptors by permuting the vertices randomly 10 times.
there will be ten folder one for each descriptors and data will be trained on each dataset individually 10 times and average of the best 10 results will be reported.

In [23]:
from joblib import Parallel, delayed 

N_JOBS = 6
connected_graphs_components    = return_connected_components(n=6, imp_feature=True)
dir_name                       = "HOSD_M_basic_l2_"

#add graph dataset names here in the list 
dataset_list = ['NCI1']
for file_name in dataset_list:
    t0 = time.time()
    graph_list, data_y, data_nodes  = return_dataset(file_name)

    for k in range(10):
    #data_cl                        = return_cluster_stats(graph_list, connected_graphs_components)
    #basic_stat                     = return_basic_stat(graph_list)
        graph_list                      =[i.permute_vertices(random.sample(i.vs.indices, len(i.vs.indices))) for i in graph_list]
        for g_ in graph_list:
            g_.vs['idx'] = [str(i) for i in g_.vs.indices]
        basic_stat                      =return_basic_stat(graph_list)
        dict_hist1, cn_g1               =zip(*Parallel(n_jobs=N_JOBS )(delayed(return_hist_and_contracted_graphs)(g, connected_graphs_components) for g in tqdm_notebook(graph_list, desc='Computing Histogram')))
        dict_hist2, cn_g2               =zip(*Parallel(n_jobs=N_JOBS )(delayed(return_hist_and_contracted_graphs_of_graphs)(graph, connected_graphs_components, save=False) for graph in tqdm_notebook(cn_g1, desc='Computing Histogram Contracted')))
        
        dict_basic_stat_cn1             =Parallel(n_jobs=N_JOBS )(delayed(return_basic_stat_contracted)(g) for g in tqdm_notebook(cn_g1, desc='Computing Stat'))
        dict_basic_stat_cn1             =pd.DataFrame(dict_basic_stat_cn1).fillna('0')
        dict_basic_stat_cn1             =dict_basic_stat_cn1.loc[:, (dict_basic_stat_cn1 != 0).any(axis=0)]

        data_basic_stat                 =pd.DataFrame(basic_stat).transpose()

        data_hist                      = pd.DataFrame({i:j for i,j in enumerate(dict_hist1)}).transpose()
        data_hist_1                    = pd.DataFrame({i:j for i,j in enumerate(dict_hist2)}).transpose()


        #saving to txt file using numpy helper function 
        #verbosity  1
        os.makedirs(dir_name+str(k), exist_ok=True)
        data_hist_1_V1  = data_hist_1[[i for i in data_hist_1.columns if not 'HOGN' in i and '-' in i ]]
        data_X          = pd.concat([data_hist_1_V1, data_hist,data_basic_stat, dict_basic_stat_cn1], axis=1).fillna(0,inplace=False).values
        data_X          = data_X[:,~np.all(data_X==0,axis=0)]
        np.savetxt(dir_name+str(k)+'/'+file_name+'_V1_.txt', data_X.astype(np.float64))
        
        #VERBOSITY 2

        data_hist_1_V2  = data_hist_1[[i for i in data_hist_1.columns if 'HOGN' in i]]
        data_X          = pd.concat([data_hist_1_V2, data_hist, data_hist_1_V1 - data_hist_1_V2, data_basic_stat, dict_basic_stat_cn1 ], axis=1).fillna(0,inplace=False).values
        data_X          = data_X[:,~np.all(data_X==0,axis=0)]
        np.savetxt(dir_name+str(k)+'/'+file_name+'_V2_.txt', data_X.astype(np.float64))

         #VERBOSITY 3
        data_hist_1_V3  = data_hist_1[[i for i in data_hist_1.columns if '-' not in i ]]
        data_X          = pd.concat([data_hist_1_V3, data_hist, data_basic_stat, dict_basic_stat_cn1, ], axis=1).fillna(0,inplace=False).values
        data_X          = data_X[:,~np.all(data_X==0,axis=0)]
        np.savetxt(dir_name+str(k)+'/'+file_name+'_V3_.txt', data_X.astype(np.float64))
        
        #SAVE LABELS
        np.savetxt(dir_name+str(k)+'/'+file_name+'_y.txt', data_y.astype(np.float64))

Extracting dataset  NCI1..
Parsing dataset  NCI1..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..


In [28]:
for file_name in dataset_list:
    for i1 in ['_V1_','_V2_','_V3_']:
        max_arr= []
        for i2,seed in zip(range(0,10),[345,567,890,5678,78, 6,1122,101,11111,42]):
            data_X = np.loadtxt(dir_name+str(i2)+'/'+file_name+i1+".txt")
            data_y = np.loadtxt(dir_name+str(i2)+'/'+file_name+'_y.txt')
            estimator  = RandomForestClassifier(criterion='gini', max_depth=None, min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                                    max_leaf_nodes=None, bootstrap=True, 
                                                    oob_score=False, n_jobs=1,verbose=0, warm_start=False,
                                                    class_weight=None)

            param_grid = {'n_estimators':[50,100,500], 'max_features':['sqrt'], 'min_samples_split':[2,3,4,5]}
            kf = StratifiedKFold(n_splits=10, random_state = seed, shuffle = True)
            grid_rf    = GridSearchCV(estimator, param_grid, scoring=None, fit_params=None, n_jobs=N_JOBS, 
                         iid=False, refit=True, cv=kf, verbose=3, pre_dispatch='n_jobs', 
                         error_score='raise-deprecating',return_train_score=False)
            #print(data_X.shape)
            grid_rf.fit(data_X, data_y)
            df = pd.DataFrame(grid_rf.cv_results_)
            max_arr.append(df['mean_test_score'].max())
            #grid_rf.cv_results_['mean_test_score']
            #Pipeline2 = Pipeline([('drop_empty_column', dropCols(frac=1.0)),('grid_rf', grid_rf)])
            #res = Pipeline2.fit(data_X, data_y)

        pathlib.Path("results/").mkdir(parents=True, exist_ok=True)
        print('Result computed for', ('/results/'+file_name+str(i1)+".txt", 'results are:',np.mean(max_arr)))
        np.savetxt('results/'+file_name +i1+'.csv', max_arr)

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  26 tasks      | elapsed:   19.3s
[Parallel(n_jobs=6)]: Done 120 out of 120 | elapsed:   57.3s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  26 tasks      | elapsed:    8.6s
[Parallel(n_jobs=6)]: Done 120 out of 120 | elapsed:   45.8s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  26 tasks      | elapsed:    6.0s
[Parallel(n_jobs=6)]: Done 120 out of 120 | elapsed:   32.7s finished


Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  26 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done 120 out of 120 | elapsed:   32.9s finished


Result computed for ('/results/NCI1_V3_.txt', 'results are:', 0.7096875867468901)
